In [1]:
%autoreload 2

In [2]:
from collections import defaultdict
import gzip
import os
import pickle
import re
import sys
import time
import typing

import numpy as np
import tqdm.notebook as tqdm

sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../src'))
from src.ast_utils import load_games_from_file, _extract_game_id
from src import room_and_object_types 

In [3]:
import openai
openai.api_key = os.getenv('OPENAI_API_KEY')

In [4]:
game_texts = list(load_games_from_file('../dsl/interactive-beta.pddl'))

In [5]:
ENVIRONMENT_PROMPT_TEMPLATE = """; The environment the following games take place in has the following objects:
; <type>: <object type> (<object name>, <object name>, ...) ...
{environment_objects}
"""
OBJECT_CATEGORY_TEMPLATE = """; {category}: {object_types}"""

CATEGORY_TO_BASE_OBJECTS = {
    room_and_object_types.BALLS: room_and_object_types.BALL,
    room_and_object_types.BLOCKS: room_and_object_types.BLOCK,
    room_and_object_types.COLORS: room_and_object_types.COLOR,
}

DEFAULT_IGNORE_CATEGORIES = [room_and_object_types.ANY_OBJECT, room_and_object_types.EMPTY_OBJECT]

def generate_environment_prompt(ignore_categories: typing.List[str] = DEFAULT_IGNORE_CATEGORIES):
    type_strings = []
    for category, types in room_and_object_types.CATEGORIES_TO_TYPES.items():
        if category in ignore_categories:
            continue
        
        category_type_strings = []
        types = list(types)
        if category in CATEGORY_TO_BASE_OBJECTS:
            category = CATEGORY_TO_BASE_OBJECTS[category]
            if category in types:
                types.remove(category)

        if category == room_and_object_types.COLOR:
            types = sorted(types)
            type_strings.append(OBJECT_CATEGORY_TEMPLATE.format(category=category, object_types=" ".join(types)))
            continue

        types_with_colors = [t for t in types if any(t.startswith(c) for c in room_and_object_types.CATEGORIES_TO_TYPES[room_and_object_types.COLORS])]
        types_without_colors = [t for t in types if t not in types_with_colors]

        for t in sorted(types_without_colors):
            t_with_colors = [tc for tc in types_with_colors if tc.endswith(t)]
            if t_with_colors:
                category_type_strings.append(f'{t} ({", ".join(sorted(t_with_colors))})')
            else:
                category_type_strings.append(t)

        type_strings.append(OBJECT_CATEGORY_TEMPLATE.format(category=category, object_types=" ".join(category_type_strings)))

    return ENVIRONMENT_PROMPT_TEMPLATE.format(environment_objects='\n'.join(type_strings))


ENVIRONMENT_PROMPT = generate_environment_prompt()
print(ENVIRONMENT_PROMPT)

; The environment the following games take place in has the following objects:
; <type>: <object type> (<object name>, <object name>, ...) ...
; agent: agent
; ball: basketball beachball dodgeball (blue_dodgeball, pink_dodgeball, red_dodgeball) golfball (green_golfball)
; block: bridge_block cube_block (blue_cube_block, tan_cube_block, yellow_cube_block) cylindrical_block flat_block pyramid_block (blue_pyramid_block, red_pyramid_block, yellow_pyramid_block) tall_cylindrical_block triangle_block
; color: blue brown gray green light_blue orange pink purple red tan white yellow
; furniture: bed blinds chair desk desk_shelf desktop drawer main_light_switch side_table top_drawer
; building: building
; large_objects: book laptop pillow teddy_bear
; orientations: sideways upright upside_down
; ramps: curved_wooden_ramp triangular_ramp (green_triangular_ramp)
; receptacles: doggie_bed hexagonal_bin
; room_features: bottom_shelf door east_sliding_door floor north_wall room_center rug shelf slid

In [6]:
GAME_START = '(define'
SETUP_SECTION = '(:setup'
PREFERENCES_SECTION = '(:constraints'
TERMINAL_SECTION = '(:terminal'
SCORING_SECTION = '(:scoring'
SECTION_KEYS = [SETUP_SECTION, PREFERENCES_SECTION, TERMINAL_SECTION, SCORING_SECTION]
OPEN_AND = '(and'
SECTION_SUFFIX = ')\n'
GAME_END_SUFFIX = '))\n'


DEFAULT_RANDOM_SEED = 33


def split_game(game_text: str, 
    start_section_str: str, end_section_str: typing.Union[None, str, typing.Sequence[str]],
    suffix: str = SECTION_SUFFIX, game_end_suffix: str = GAME_END_SUFFIX) -> typing.Tuple[str, str]:
    start_index = game_text.find(start_section_str)

    if end_section_str is None:
        end_index = None

    elif isinstance(end_section_str, str):
        end_index = game_text.find(end_section_str)

    else:
        end_indices = [game_text.find(s) for s in end_section_str]
        end_index = min([i for i in end_indices if i >= 0])

    if start_index == -1:
        start_index = end_index

    if end_index == -1:
        end_index = None

    if end_index is None:
        return game_text[:start_index] + start_section_str, game_end_suffix

    return game_text[:start_index] + start_section_str, suffix + game_text[end_index:]
        
        
def create_multi_game_prompt_suffix(game_texts: typing.List[str],
    n_games_before: int, n_games_after: int, 
    target_game_split_func: typing.Callable[..., typing.Tuple[str, str]],
    target_game_split_func_args: typing.Dict[str, typing.Any],
    game_filter_str: typing.Optional[str] = None, include_environment_prompt: bool = True,
    rng: typing.Optional[np.random.Generator] = None, random_seed: int = DEFAULT_RANDOM_SEED
    ) -> typing.Tuple[str, str]:

    if rng is None:
        rng = np.random.default_rng(random_seed)

    if game_filter_str is not None:
        game_texts = [g for g in game_texts if game_filter_str in g]

    indices = rng.choice(len(game_texts), n_games_before + n_games_after + 1, replace=False)
    before_indices = indices[:n_games_before]
    target_index = indices[n_games_before]
    after_indices = indices[n_games_before + 1:]

    return create_multi_game_prompt_suffix_from_indices(game_texts, target_game_split_func, target_game_split_func_args, before_indices, target_index, after_indices, 
                                                        include_environment_prompt=include_environment_prompt)


def create_multi_game_prompt_suffix_from_indices(game_texts: typing.List[str], 
    target_game_split_func: typing.Callable[..., typing.Tuple[str, str]], 
    target_game_split_func_args: typing.Dict[str, typing.Any], 
    before_indices: typing.Sequence[int], 
    target_index: int, after_indices: typing.Sequence[int],
    include_environment_prompt: bool = True):

    before_games = [game_texts[i] for i in before_indices]
    target_prompt, target_suffix = target_game_split_func(game_texts[target_index], **target_game_split_func_args)
    after_games = [game_texts[i] for i in after_indices]

    before_games.append(target_prompt)
    after_games.insert(0, target_suffix)

    if include_environment_prompt:
        before_games.insert(0, ENVIRONMENT_PROMPT)

    return '\n'.join(before_games), '\n'.join(after_games)


GAMES_WITH_SETUP = set([i for i, g in enumerate(game_texts) if SETUP_SECTION in g])
GAMES_WITH_SETUP_LIST = list(GAMES_WITH_SETUP)
GAMES_WITH_TERMINAL = set([i for i, g in enumerate(game_texts) if TERMINAL_SECTION in g])
GAMES_WITH_TERMINAL_LIST = list(GAMES_WITH_TERMINAL)


def create_all_prompts_for_game(game_texts: typing.List[str], 
    game_index: int, n_games_before: int, n_games_after: int, 
    rng: typing.Optional[np.random.Generator] = None, random_seed: int = DEFAULT_RANDOM_SEED) -> typing.Dict[str, typing.Any]:

    if rng is None:
        rng = np.random.default_rng(random_seed)

    setup_games = GAMES_WITH_SETUP_LIST[:]
    if game_index in setup_games:
        setup_games.remove(game_index)

    preferences_and_scoring_games = list(range(len(game_texts)))
    preferences_and_scoring_games.remove(game_index)

    terminal_games = GAMES_WITH_TERMINAL_LIST[:]
    if game_index in terminal_games:
        terminal_games.remove(game_index)

    setup_context_indices = rng.choice(setup_games, n_games_before + n_games_after, replace=False)
    preference_context_indices = rng.choice(preferences_and_scoring_games, n_games_before + n_games_after, replace=False)
    terminal_context_indices = rng.choice(terminal_games, n_games_before + n_games_after, replace=False)
    scoring_context_indices = rng.choice(preferences_and_scoring_games, n_games_before + n_games_after, replace=False)

    setup_prompt, setup_suffix = create_multi_game_prompt_suffix_from_indices(game_texts, split_game, 
        {'start_section_str': SETUP_SECTION, 'end_section_str': PREFERENCES_SECTION}, 
        setup_context_indices[:n_games_before], game_index, setup_context_indices[n_games_before:])  # type: ignore

    preference_prompt, preference_suffix = create_multi_game_prompt_suffix_from_indices(game_texts, split_game, 
            {'start_section_str': PREFERENCES_SECTION, 'end_section_str': (TERMINAL_SECTION, SCORING_SECTION)},
            preference_context_indices[:n_games_before], game_index, preference_context_indices[n_games_before:])  # type: ignore

    terminal_prompt, terminal_suffix = create_multi_game_prompt_suffix_from_indices(game_texts, split_game, 
            {'start_section_str': TERMINAL_SECTION, 'end_section_str': SCORING_SECTION},
            terminal_context_indices[:n_games_before], game_index, terminal_context_indices[n_games_before:])  # type: ignore

    scoring_prompt, scoring_suffix = create_multi_game_prompt_suffix_from_indices(game_texts, split_game, 
            {'start_section_str': SCORING_SECTION, 'end_section_str': None},
            scoring_context_indices[:n_games_before], game_index, scoring_context_indices[n_games_before:])  # type: ignore

    return {
        SETUP_SECTION: dict(prompt=setup_prompt, suffix=setup_suffix, before_indices=setup_context_indices[:n_games_before], 
            target_index=game_index, after_indices=setup_context_indices[n_games_before:]),  
        PREFERENCES_SECTION: dict(prompt=preference_prompt, suffix=preference_suffix, before_indices=preference_context_indices[:n_games_before], 
            target_index=game_index, after_indices=preference_context_indices[n_games_before:]),
        TERMINAL_SECTION: dict(prompt=terminal_prompt, suffix=terminal_suffix, before_indices=terminal_context_indices[:n_games_before],
            target_index=game_index, after_indices=terminal_context_indices[n_games_before:]),
        SCORING_SECTION: dict(prompt=scoring_prompt, suffix=scoring_suffix, before_indices=scoring_context_indices[:n_games_before],
            target_index=game_index, after_indices=scoring_context_indices[n_games_before:])
    }



def create_iterative_prompts_for_game(game_texts: typing.List[str], 
    game_index: int, n_games_before: int, n_games_after: int, 
    rng: typing.Optional[np.random.Generator] = None, random_seed: int = DEFAULT_RANDOM_SEED) -> typing.Dict[str, typing.Any]:

    if rng is None:
        rng = np.random.default_rng(random_seed)

    preferences_and_scoring_games = list(range(len(game_texts)))
    preferences_and_scoring_games.remove(game_index)

    context_indices = rng.choice(preferences_and_scoring_games, n_games_before + n_games_after, replace=False)

    setup_prompt, setup_suffix = create_multi_game_prompt_suffix_from_indices(game_texts, split_game, 
        {'start_section_str': SETUP_SECTION, 'end_section_str': PREFERENCES_SECTION}, 
        context_indices[:n_games_before], game_index, context_indices[n_games_before:])  # type: ignore

    preference_prompt, preference_suffix = create_multi_game_prompt_suffix_from_indices(game_texts, split_game, 
            {'start_section_str': PREFERENCES_SECTION, 'end_section_str': (TERMINAL_SECTION, SCORING_SECTION)},
            context_indices[:n_games_before], game_index, context_indices[n_games_before:])  # type: ignore

    terminal_prompt, terminal_suffix = create_multi_game_prompt_suffix_from_indices(game_texts, split_game, 
            {'start_section_str': TERMINAL_SECTION, 'end_section_str': SCORING_SECTION},
            context_indices[:n_games_before], game_index, context_indices[n_games_before:])  # type: ignore

    scoring_prompt, scoring_suffix = create_multi_game_prompt_suffix_from_indices(game_texts, split_game, 
            {'start_section_str': SCORING_SECTION, 'end_section_str': None},
            context_indices[:n_games_before], game_index, context_indices[n_games_before:])  # type: ignore

    return {
        SETUP_SECTION: dict(prompt=setup_prompt, suffix=setup_suffix, before_indices=context_indices[:n_games_before], 
            target_index=game_index, after_indices=context_indices[n_games_before:]),  
        PREFERENCES_SECTION: dict(prompt=preference_prompt, suffix=preference_suffix, before_indices=context_indices[:n_games_before], 
            target_index=game_index, after_indices=context_indices[n_games_before:]),
        TERMINAL_SECTION: dict(prompt=terminal_prompt, suffix=terminal_suffix, before_indices=context_indices[:n_games_before],
            target_index=game_index, after_indices=context_indices[n_games_before:]),
        SCORING_SECTION: dict(prompt=scoring_prompt, suffix=scoring_suffix, before_indices=context_indices[:n_games_before],
            target_index=game_index, after_indices=context_indices[n_games_before:])
    }



In [7]:
DEFAULT_CODEX_MODEL = "code-davinci-002"
DEFAULT_TEMPERATURE = 0.67
DEFAULT_MAX_TOKENS = 512
DEFAULT_STOP_SEQUENCES = [GAME_START, PREFERENCES_SECTION, TERMINAL_SECTION, SCORING_SECTION]
MAX_N = 10


DEFAULT_COMPLETION_KWARGS = dict(
    model=DEFAULT_CODEX_MODEL,
    temperature=DEFAULT_TEMPERATURE,
    max_tokens=DEFAULT_MAX_TOKENS,
    stop=DEFAULT_STOP_SEQUENCES,
)


def generate_codex_completions(prompt: str, suffix: str, n: int, 
    completion_kwargs: typing.Optional[typing.Dict[str, typing.Any]],
    ):

    if n > MAX_N:
        raise ValueError(f'n must be <= {MAX_N}')

    if completion_kwargs is None:
        completion_kwargs = DEFAULT_COMPLETION_KWARGS

    else:
        kwargs = DEFAULT_COMPLETION_KWARGS.copy()
        kwargs.update(completion_kwargs)
        completion_kwargs = kwargs

    completion_kwargs['prompt'] = prompt
    completion_kwargs['suffix'] = suffix
    completion_kwargs['n'] = n

    return openai.Completion.create(**completion_kwargs)
    

N_AUGMENTATIONS_PER_SECTION = {
    SETUP_SECTION: 3,
    PREFERENCES_SECTION: 6,
    TERMINAL_SECTION: 3,
    SCORING_SECTION: 3,
}

def generate_single_game_augmentations(game_texts: typing.List[str], 
    game_index: int, n_games_before: int, n_games_after: int, 
    rng: typing.Optional[np.random.Generator] = None, random_seed: int = DEFAULT_RANDOM_SEED,
    completion_kwargs_by_section: typing.Optional[typing.Dict[str, typing.Dict[str, typing.Any]]] = None,
    sleep_between_requests_seconds: float = 0.0,
    ):

    if completion_kwargs_by_section is None:
        completion_kwargs_by_section = defaultdict(dict)

    prompts_by_section = create_all_prompts_for_game(game_texts, game_index, n_games_before, n_games_after, rng, random_seed)
    
    for section in prompts_by_section:
        prompt = prompts_by_section[section]['prompt']
        suffix = prompts_by_section[section]['suffix']
        prompts_by_section[section]['results'] = generate_codex_completions(prompt, suffix, N_AUGMENTATIONS_PER_SECTION[section], completion_kwargs_by_section[section])
        time.sleep(sleep_between_requests_seconds)

    prompts_by_section['game_index'] = game_index
    prompts_by_section['game_id'] = _extract_game_id(game_texts[game_index])

    return prompts_by_section


def _iterative_generation_single_section(section_dict: typing.Dict[str, typing.Any],
    n_completions: int = 1, section_completions_kwarg: typing.Optional[typing.Dict[str, typing.Any]] = None):

    # TODO: handle `n_completions > 1`
    if n_completions > 1:
        raise NotImplementedError('n_completions > 1 not yet implemented')

    section_dict['results'] = generate_codex_completions(section_dict['prompt'], section_dict['suffix'], 
        n_completions, section_completions_kwarg)
    section_dict['completion'] = section_dict['results'].choices[0].text


def iterative_single_game_generation(game_texts: typing.List[str], 
    game_index: int, n_games_before: int, n_games_after: int, 
    rng: typing.Optional[np.random.Generator] = None, random_seed: int = DEFAULT_RANDOM_SEED,
    completion_kwargs_by_section: typing.Optional[typing.Dict[str, typing.Dict[str, typing.Any]]] = None,
    sleep_between_requests_seconds: float = 0.0,
    ):

    if completion_kwargs_by_section is None:
        completion_kwargs_by_section = defaultdict(dict)

    prompts_by_section = create_iterative_prompts_for_game(game_texts, game_index, n_games_before, n_games_after, rng, random_seed)

    sections = list(prompts_by_section.keys())
    sections.append(None)

    for current_section, next_section in zip(sections[:-1], sections[1:]):
        section_dict = prompts_by_section[current_section]
        _iterative_generation_single_section(section_dict, 1, completion_kwargs_by_section[current_section])

        if next_section is not None:
            next_updated_prompt = ' '.join((section_dict['prompt'], section_dict['completion']))
            while next_updated_prompt.count('(') > next_updated_prompt.count(')'):
                next_updated_prompt += SECTION_SUFFIX
            else:
                next_updated_prompt += '\n'
        
            next_updated_prompt += next_section
            prompts_by_section[next_section]['prompt'] = next_updated_prompt
            time.sleep(sleep_between_requests_seconds)

        else:
            final_generation_with_prompt = ' '.join((section_dict['prompt'], section_dict['completion']))
            final_generation = final_generation_with_prompt[final_generation_with_prompt.rfind(GAME_START):]
            while final_generation.count('(') > final_generation.count(')'):
                final_generation += ')'

            prompts_by_section['final_generation'] = final_generation
    
    prompts_by_section['game_index'] = game_index
    prompts_by_section['game_id'] = _extract_game_id(game_texts[game_index])

    return prompts_by_section
    

def LCS(s1: str, s2: str) -> typing.Tuple[int, int]:
    m = np.zeros((len(s1) + 1, len(s2) + 1), dtype=int)
    longest, x_longest = 0, 0
    for x in range(1, 1 + len(s1)):
        for y in range(1, 1 + len(s2)):
            if s1[x - 1] == s2[y - 1]:
                m[x][y] = m[x - 1][y - 1] + 1
                if m[x][y] > longest:
                    longest = m[x][y]
                    x_longest = x
            else:
                m[x][y] = 0
                
    return x_longest - longest, longest


In [8]:
random_seed = 121
output_path = f'/Users/guydavidson/projects/game-generation-modeling/dsl/codex/codex_augmentations_seed_{random_seed}.pkl.gz'
sleep_interval = 30
games_before = 5
games_after = 0
results_per_game = []


In [9]:
time.sleep(sleep_interval)
results_by_section = generate_single_game_augmentations(game_texts, 3, games_before, games_after, 
                random_seed=random_seed, sleep_between_requests_seconds=sleep_interval)

InvalidRequestError: The model: `code-davinci-002` does not exist

In [40]:
for section in results_by_section:
    r = results_by_section[section]
    print(f'{section}:')
    WHITESPSACE_PATTERN = re.compile(r'\s+')
    prompt = WHITESPSACE_PATTERN.sub(' ', r['prompt'])
    texts = [WHITESPSACE_PATTERN.sub(' ', c.text) for c in r['results'].choices]

    lcs_results = [LCS(WHITESPSACE_PATTERN.sub(' ', text), prompt) for text in texts]
    for i, text in enumerate(texts):
        print(f'{i}:')
        print(f'    {lcs_results[i]} ({lcs_results[i][1] / len(text)* 100:.2f}%)')
        print(f'    {text[lcs_results[i][0]:lcs_results[i][0]+lcs_results[i][1]]}')
        print()

    print()

(:setup:
0:
    (124, 32) (13.79%)
    )))) (exists (?h - hexagonal_bin

1:
    (0, 36) (42.35%)
     (and (exists (?h - hexagonal_bin) (

2:
    (86, 58) (21.48%)
     dodgeball triangle_block pyramid_block)) (game-optional (


(:constraints:
0:
    (109, 88) (40.55%)
    ))) (hold (and (in_motion ?d) (not (agent_holds ?d)))) (once (and (not (in_motion ?d)) (

1:
    (0, 405) (100.00%)
     (and (preference dodgeballsInPlace (exists (?d - dodgeball ?h - hexagonal_bin ?w1 ?w2 - wall) (at-end (and (in ?h ?d) (adjacent ?h ?w1) (adjacent ?h ?w2))) ) ) (preference blocksInPlace (exists (?c - cube_block ?s - shelf) (at-end (on ?s ?c)) ) ) (preference smallItemsInPlace (exists (?o - (either cellphone key_chain mug credit_card cd watch alarm_clock) ?d - drawer) (at-end (and (in ?d ?o) )) ) ) )) 

2:
    (55, 173) (75.88%)
    ?h - hexagonal_bin) (then (once (and (agent_holds ?b) (on rug agent))) (hold (and (in_motion ?b) (not (agent_holds ?b)))) (once (and (not (in_motion ?b)) (in ?h ?b))) ) 

TypeError: 'int' object is not subscriptable

In [ ]:
done = False
while not done:
    try:
        last_index_found = results_per_game[-1]['game_index'] if len(results_per_game) > 0 else -1
        for index in tqdm.tnrange(last_index_found + 1, len(game_texts)):
            print(index)
            results_by_section = generate_single_game_augmentations(game_texts, index, games_before, games_after, 
                random_seed=random_seed, sleep_between_requests_seconds=sleep_interval)
            results_per_game.append(results_by_section)
            time.sleep(sleep_interval)
        done = True
    except openai.error.RateLimitError as e:
        print(f'Rate limit error: {e}, sleeping for {sleep_interval} seconds')
        time.sleep(sleep_interval)

original_output_path = output_path[:]
i = 0
while os.path.exists(output_path):
    output_path = original_output_path + f'_{i}'
    i += 1

with gzip.open(output_path, 'wb') as f:
    pickle.dump(results_per_game, f, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
random_seed = 11
output_path = f'/Users/guydavidson/projects/game-generation-modeling/dsl/codex/codex_single_game_iterative_seed_{random_seed}.pkl.gz'
sleep_interval = 10
games_before = 4
games_after = 1
iterative_results_per_game = []


In [10]:
sleep_interval = 15
done = False
while not done:
    try:
        last_index_found = iterative_results_per_game[-1]['game_index'] if len(iterative_results_per_game) > 0 else -1
        for index in tqdm.tnrange(last_index_found + 1, len(game_texts)):
            print(index)
            results_by_section = iterative_single_game_generation(game_texts, index, games_before, games_after, 
                random_seed=random_seed, sleep_between_requests_seconds=sleep_interval)
            iterative_results_per_game.append(results_by_section)
            time.sleep(sleep_interval)
        done = True
    except openai.error.RateLimitError as e:
        print(f'Rate limit error: {e}, sleeping for {sleep_interval} seconds')
        time.sleep(sleep_interval)
    except openai.error.Timeout as e:
        print(f'Timeout: {e}, sleeping for {sleep_interval} seconds')
        time.sleep(sleep_interval)
    

original_output_path = output_path[:]
i = 0
while os.path.exists(output_path):
    output_path = original_output_path + f'_{i}'
    i += 1

with gzip.open(output_path, 'wb') as f:
    pickle.dump(iterative_results_per_game, f, protocol=pickle.HIGHEST_PROTOCOL)

  0%|          | 0/92 [00:00<?, ?it/s]

6
Rate limit error: Rate limit reached for default-code-davinci-002 in organization org-Gkt74uwdfaydquT2epfvcVp1 on requests per min. Limit: 20.000000 / min. Current: 30.000000 / min. Contact support@openai.com if you continue to have issues., sleeping for 15 seconds


  0%|          | 0/92 [00:00<?, ?it/s]

6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
Rate limit error: Rate limit reached for default-code-davinci-002 in organization org-Gkt74uwdfaydquT2epfvcVp1 on requests per min. Limit: 20.000000 / min. Current: 30.000000 / min. Contact support@openai.com if you continue to have issues., sleeping for 15 seconds


  0%|          | 0/55 [00:00<?, ?it/s]

...

48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
Rate limit error: Rate limit reached for default-code-davinci-002 in organization org-Gkt74uwdfaydquT2epfvcVp1 on requests per min. Limit: 20.000000 / min. Current: 30.000000 / min. Contact support@openai.com if you continue to have issues., sleeping for 15 seconds


  0%|          | 0/28 [00:00<?, ?it/s]

70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
Rate limit error: Rate limit reached for default-code-davinci-002 in organization org-Gkt74uwdfaydquT2epfvcVp1 on requests per min. Limit: 20.000000 / min. Current: 30.000000 / min. Contact support@openai.com if you continue to have issues., sleeping for 15 seconds


  0%|          | 0/12 [00:00<?, ?it/s]

86
87
88
89
90
91
92
93
94
95
96
97


In [ ]:
with gzip.open(output_path, 'wb') as f:
    pickle.dump(iterative_results_per_game, f, protocol=pickle.HIGHEST_PROTOCOL)

# What's between these and a working augmented dataset:

1. Minor sanity checks, e.g. that the number of parentheses matches (or otherwise, remove trailing close parentheses)
2. Decide what to do about productions truncated for length (probably remove last partial preference)
3. Decide on a procedure to create games (sample from within the productions for a given game? between productions for differeng games?)
4. Generate a dataset with the procedure I have in mind, verify that the ASTs parse
5. Run it through the fitness function, compare the results to the real data
6. Create regrown corruptions for it, and repeat the previous synthetic experiments.


In [ ]:
import pickle
with open(f'/Users/guydavidson/Downloads/test.pkl', 'wb') as pickle_file:
    pickle.dump(results_by_section, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)

In [18]:
prompt_with_comments = """
(define (game game-23) (:domain few-objects-room-v1)  ; 23

(:constraints (and 
    ; count dodgeball throws that succesfully land in the bin
    (preference throwBallToBin
        (exists (?d - dodgeball ?h - hexagonal_bin)
            (then 
                (once (agent_holds ?d))
                (hold (and (not (agent_holds ?d)) (in_motion ?d))) 
                (once (and (not (in_motion ?d)) (in ?h ?d)))
            )
        )
    )
    ; count any throw attempt
    (preference throwAttempt
        (exists (?d - dodgeball)
            (then 
                (once (agent_holds ?d))
                (hold (and (not (agent_holds ?d)) (in_motion ?d))) 
                (once (not (in_motion ?d)))
            )
        )
    )
))
(:scoring (+
    ; one point for each throw that lands in the bin
    (count throwBallToBin)
    ; subtract 1/5 point for each throw attempt
    (- (/ (count throwAttempt) 5))
)))

(define (game game-43) (:domain medium-objects-room-v1)  ; 43
(:setup 
; place the doggie bed within 1 unit of the room center
(and 
    (exists (?d - doggie_bed) (game-conserved (< (distance room_center ?d) 1)))
))
(:constraints (and 
    ; for any type of ball
    (forall (?b - ball) (and 
        ; throw the ball onto the doggie bed without it touching any walls
        (preference throwBallToDoggieBed
            (exists (?d - doggie_bed)
                (then 
                    (once (agent_holds ?b))
                    (hold (and (not (agent_holds ?b)) (in_motion ?b) (not (exists (?w - wall) (touch ?w ?b ))))) 
                    (once (and (not (in_motion ?b)) (on ?d ?b)))
                )
            )
        )
        ; throw the ball onto the doggie bed while it touches a wall
        (preference throwBallToDoggieBedOffWall
            (exists (?d - doggie_bed ?w - wall)
                (then 
                    (once (agent_holds ?b))
                    (hold-while 
                        (and (not (agent_holds ?d)) (in_motion ?b))
                        (touch ?w ?b)    
                    ) 
                    (once (and (not (in_motion ?b)) (on ?d ?b)))
                )
            )
        )  
    ))
))
(:scoring (+
    ; one point for each throw onto the doggie bed with a basketball
    (count throwBallToDoggieBed:basketball)
    ; two points for each throw onto the doggie bed with a beachball
    (* 2 (count throwBallToDoggieBed:beachball))
    ; three points for each throw onto the doggie bed with a dodgeball
    (* 3 (count throwBallToDoggieBed:dodgeball))
    ; two points for each throw onto the doggie bed with a basketball while it touches a wall
    (* 2 (count throwBallToDoggieBedOffWall:basketball))
    ; three points for each throw onto the doggie bed with a beachball while it touches a wall
    (* 3 (count throwBallToDoggieBedOffWall:beachball))
    ; four points for each throw onto the doggie bed with a dodgeball while it touches a wall
    (* 4 (count throwBallToDoggieBedOffWall:dodgeball))
)))

(define (game game-51) (:domain few-objects-room-v1)  ; 51

(:constraints (and 
    ; count dodgeball throws that succesfully land in the bin
    (preference throwToBin
        (exists (?d - dodgeball ?h - hexagonal_bin)
            (then 
                (once (agent_holds ?d))
                (hold (and (not (agent_holds ?d)) (in_motion ?d))) 
                (once (and (not (in_motion ?d)) (in ?h ?d)))
            )
        )
    )
))
(:scoring 
    ; one point for each throw that lands in the bin
    (count throwToBin)  
))
(define (game game-52) (:domain few-objects-room-v1)  ; 52
(:setup 
; place the hexagonal bin within 1.5 units of the desk
"""

suffix_for_comments = """
)
(:constraints (and <constraints>))
(:scoring <scoring>)
)
"""

def fix_parentheses(text: str) -> str:
    while text.count('(') > text.count(')'):
        text += ')'

    while text.count(')') > text.count('('):
        last_close_paren = text.rfind(')')
        text = text[:last_close_paren] + text[last_close_paren+1:]

    return text


In [9]:
completions = generate_codex_completions(prompt_with_comments, suffix_for_comments, 3, DEFAULT_COMPLETION_KWARGS)

In [15]:
print(completions.choices)
setup_text = completions.choices[0].text

[<OpenAIObject at 0x11b9fa390> JSON: {
  "finish_reason": "stop",
  "index": 0,
  "logprobs": null,
  "text": "(and \n    (exists (?d - hexagonal_bin ?de - desk) (game-conserved (< (distance ?d ?de) 1.5)))\n))\n"
}, <OpenAIObject at 0x11b9fa5c0> JSON: {
  "finish_reason": "stop",
  "index": 1,
  "logprobs": null,
  "text": "(and \n    (exists (?h - hexagonal_bin ?d - desk) (game-conserved (< (distance ?h ?d) 1.5)))\n))\n"
}, <OpenAIObject at 0x11ba113f0> JSON: {
  "finish_reason": "stop",
  "index": 2,
  "logprobs": null,
  "text": "(and \n    (exists (?b - hexagonal_bin ?d - desk) (game-conserved (< (distance ?b ?d) 1.5)))\n))\n"
}]


In [23]:
first_preference_components = [
    prompt_with_comments, 
    fix_parentheses(setup_text),
    ')',
    '(:constraints (and',
    '    ; count baseketball throws that succesfully land in the bin',
]
prompt_for_first_preference = '\n'.join(first_preference_components)

suffix_for_first_preference = """

    ; count golfball throws that succesfully land in the bin after touching the desk
    <preference>
))
(:scoring <scoring>)
)
"""

completions = generate_codex_completions(prompt_for_first_preference, suffix_for_first_preference, 3, DEFAULT_COMPLETION_KWARGS)

In [22]:
for choice in completions.choices:
    print(choice.index)
    print(choice.text)

0

    (preference throwToBin:basketball
        (exists (?b - basketball ?h - hexagonal_bin)
            (then 
                (once (agent_holds ?b))
                (hold (and (not (agent_holds ?b)) (in_motion ?b))) 
                (once (and (not (in_motion ?b)) (in ?h ?b)))
            )
        )
    )
    ; count baseketball throws that succesfully land in the bin after touching the desk
    (preference throwToBinOffDesk:basketball
        (exists (?b - basketball ?h - hexagonal_bin ?de - desk)
            (then 
                (once (agent_holds ?b))
                (hold-while 
                    (and (not (agent_holds ?b)) (in_motion ?b))
                    (touch ?de ?b)    
                ) 
                (once (and (not (in_motion ?b)) (in ?h ?b)))
            )
        )
    )
    ; count dodgeball throws that succesfully land in the bin
    (preference throwToBin:dodgeball
        (exists (?d - dodgeball ?h - hexagonal_bin)
            (then 
                (onc

In [24]:
first_preference = """
    (preference throwToBin
        (exists (?b - basketball ?h - hexagonal_bin)
            (then 
                (once (agent_holds ?b))
                (hold (and (not (agent_holds ?b)) (in_motion ?b))) 
                (once (and (not (in_motion ?b)) (in ?h ?b)))
            )
        )
    )
"""

second_preference_components = [
    prompt_for_first_preference,
    fix_parentheses(first_preference),
    '; count golfball throws that succesfully land in the bin after touching the desk',
]
prompt_for_second_preference = '\n'.join(second_preference_components)
suffix_for_second_preference = """
))
(:scoring <scoring>)
)
"""

completions = generate_codex_completions(prompt_for_second_preference, suffix_for_second_preference, 3, DEFAULT_COMPLETION_KWARGS)

In [29]:
print(completions.choices)
second_preference = fix_parentheses(completions.choices[0].text)
print(second_preference)

[<OpenAIObject at 0x11b5ec590> JSON: {
  "finish_reason": "stop",
  "index": 0,
  "logprobs": null,
  "text": "\n\n    (preference throwToBinOffDesk\n        (exists (?g - golfball ?h - hexagonal_bin ?de - desk)\n            (then \n                (once (agent_holds ?g))\n                (hold-while \n                    (and (not (agent_holds ?g)) (in_motion ?g))\n                    (touch ?de ?g)\n                )\n                (once (and (not (in_motion ?g)) (in ?h ?g)))\n            )\n        )\n    )\n))\n"
}, <OpenAIObject at 0x11b9fa570> JSON: {
  "finish_reason": "stop",
  "index": 1,
  "logprobs": null,
  "text": "\n\n    (preference throwToBinTouchDesk\n        (exists (?g - golfball ?h - hexagonal_bin ?de - desk)\n            (then \n                (once (agent_holds ?g))\n                (hold-while \n                    (and (not (agent_holds ?g)) (in_motion ?g))\n                    (touch ?de ?g)    \n                ) \n                (once (and (not (in_motion

In [30]:
scoring_components = [
    prompt_for_second_preference,
    second_preference,
    '))',
    '(:scoring',
    '    ; one point for each throw that lands in the bin',
    '    ; two points for each throw that lands in the bin after touching the desk',
]

prompt_for_scoring = '\n'.join(scoring_components)
suffix_for_scoring = """
))
"""

completions = generate_codex_completions(prompt_for_scoring, suffix_for_scoring, 3, DEFAULT_COMPLETION_KWARGS)

In [32]:
print(completions.choices)
scoring = fix_parentheses(completions.choices[0].text)
print(scoring)

[<OpenAIObject at 0x136aa7470> JSON: {
  "finish_reason": "stop",
  "index": 0,
  "logprobs": null,
  "text": "\n    (+ (count throwToBin) (* 2 (count throwToBinOffDesk)))\n))\n"
}, <OpenAIObject at 0x11be7b470> JSON: {
  "finish_reason": "stop",
  "index": 1,
  "logprobs": null,
  "text": "\n    (+ (count throwToBin) (* 2 (count throwToBinOffDesk)))\n))\n\n"
}, <OpenAIObject at 0x11ba927f0> JSON: {
  "finish_reason": "stop",
  "index": 2,
  "logprobs": null,
  "text": "\n    (+\n    (count throwToBin)\n    (* 2 (count throwToBinOffDesk))"
}]

    (+ (count throwToBin) (* 2 (count throwToBinOffDesk)))




In [34]:
final_text = prompt_for_scoring + scoring + '))'
final_game = final_text[final_text.rfind('(define'):]
print(final_game)

(define (game 602d84f17cdd707e9caed37a-52) (:domain few-objects-room-v1)  ; 52
(:setup 
; place the hexagonal bin within 1.5 units of the desk

(and 
    (exists (?d - hexagonal_bin ?de - desk) (game-conserved (< (distance ?d ?de) 1.5)))
)

)
(:constraints (and
    ; count baseketball throws that succesfully land in the bin

    (preference throwToBin
        (exists (?b - basketball ?h - hexagonal_bin)
            (then 
                (once (agent_holds ?b))
                (hold (and (not (agent_holds ?b)) (in_motion ?b))) 
                (once (and (not (in_motion ?b)) (in ?h ?b)))
            )
        )
    )

; count golfball throws that succesfully land in the bin after touching the desk


    (preference throwToBinOffDesk
        (exists (?g - golfball ?h - hexagonal_bin ?de - desk)
            (then 
                (once (agent_holds ?g))
                (hold-while 
                    (and (not (agent_holds ?g)) (in_motion ?g))
                    (touch ?de ?g)
        